In [1]:
using Images
using PyPlot
using Statistics
using LsqFit
using DelimitedFiles

#setting the correct GUI backend for proper figure display
include("utils.jl")
set_gui();

In [2]:
#provide a path to the kymograph
img_path = "example/"
img_name = "example.tif"
loadpath = string(img_path,img_name);

timestep = 2;  #dt between kymograph lines in seconds
mag = 0.129;   #pixel size in micrometers

#which channel of the kymograph contains moving particles to trace?
channel = 1;

#load the full tiff and display the channel we will be analyzing
img = Float32.(load(loadpath));
img_a = Float32.(img);
kymograph = copy(img_a[:,:,channel]);
imshow(kymograph);

In [3]:
#manually select a portion of the kymograph containing only one moving spot
#click twice on the kymograph: once on top left corner of our future crop, once on bottom right corner
coords = zeros(0);
coords = ginput_routine(2,kymograph);

In [4]:
#display the cropped kymograph defined by our clicks
crop = kymograph[coords[1]:coords[2],coords[3]:coords[4]];
figure();
imshow(crop);

In [5]:
#record background (BG) intensity
#click twice on the cropped kymograph selecting a rectangle without moving particles
BG_coords = ginput_routine(2,crop)

(114, 182, 19, 25)

In [6]:
#record initial intensity and initial position of the moving particle
#click twice on the cropped kymograph selecting a rectangle containing >=10 initial positions of the moving particle
peak_coords = ginput_routine(2,crop)

(2, 42, 10, 19)

In [7]:
#prepare initial values for the spot fitting function  
d = mean(crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]);                #mean background intensity
dsd = stdm((crop[BG_coords[1]:BG_coords[2],BG_coords[3]:BG_coords[4]]), d);         #background SD
a = maximum(crop[peak_coords[1]:peak_coords[2],peak_coords[3]:peak_coords[4]]) - d; #peak height
b = (peak_coords[3] + peak_coords[4])/2;                                            #peak position
c = - 2;   width_tol = 0.2;                                                         #peak width and its tolerance
t = size(crop,1);                                                                   #number of timepoints (kymograph lines)

#preallocating fitting result arrays for speed
timesec = zeros(t);
timepix = zeros(t);
pixel = zeros(t);
position = zeros(t);
intensity = zeros(t);
peakwidth = zeros(t);

In [8]:
#we fit each line in our cropped kymograph with a gaussian peak of height a and width c at position b
let a = a, b = b, c = c, d = d, dsd = dsd, t = t, width_tol = width_tol
    for i in 1:t
        ydata = crop[i,:];
        xdata = collect(1:length(ydata));
        
        (a, b, c, d) = fit_gaussian(xdata, ydata, a, b, c, d, dsd)

        
        timepix[i] = i;
        pixel[i] = b;
        timesec[i] = i*timestep;
        position[i] = b*mag;
        peakwidth[i] = -c;
        
        if i > 1
            if peakwidth[i] > peakwidth[1]*(1+width_tol)
                peakwidth[i]=peakwidth[1]*(1+width_tol);
            elseif peakwidth[i] < peakwidth[1]*(1-width_tol)
                peakwidth[i]=peakwidth[1]*(1-width_tol);
            end
        end
        
        intensity[i] = a*peakwidth[i];
        
        IJulia.clear_output(true);
        println(string("Processing line ", i, " of ", t, "; a = ", round(a), "; b = ", round(b)))
    end
end

Processing line 236 of 236; a = 1186.0; b = 12.0


In [9]:
#plot fitted coordinates
fig = figure(img_name, figsize=(4,8));
subplot(121, title="cropped kymograph", xlabel="position", ylabel="time");
imshow(crop);

subplot(122, title="fitted coordinates", xlabel="position", ylabel="time");
imshow(crop);
ylim(t,0);
xlim(0,length(crop[1,:]));
grid("on");
pl = scatter(pixel, timepix, facecolor="none", edgecolors="red", s = 30);
PyPlot.title(img_name);

In [10]:
fig2 = figure(figsize=(8,5));
xlabel("time (s)");
font1 = Dict("color"=>"blue");
ylabel("position (micrometer)",fontdict=font1);
p2 = plot(timesec,position, color="blue");
xlim(0, maximum(timesec));
ylim(0, maximum(position));
ax = gca()

fig2.subplots_adjust(right=0.85);
ax2 = ax.twinx();
font2 = Dict("color"=>"orange");
ylabel("intensity (a.u.)",fontdict=font2);
p3 = plot(timesec,intensity, color="orange");
ylim(0, maximum(intensity));

PyPlot.title(img_name);

In [11]:
let savepath = string(img_path, "traced/")
    if isdir(savepath) == false
        mkdir(savepath)
    end 
end

In [12]:
function fcheck()
    filecount = 1;
    savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    while isfile(savename) == true
        filecount = filecount + 1
        savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt");
    end
    return filecount
end

filecount = fcheck();

savename = string(img_path, "traced/", img_name[1:end-4], "-", filecount, ".txt")
open(savename, "w") do io;
    writedlm(io, [timesec position intensity]);
end

subcropname = string(img_path, "traced/", img_name[1:end-4], "-", filecount, "_subcrop.tif");
save(subcropname, colorview(Gray, crop));